# Consumer Behavior Analytics - Data Cleaning

In [1]:
# Basic libraries
import pandas as pd
import numpy as np

# Utils
# import lux

# DataViz libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Data libraries
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.impute import KNNImputer

# Notebook setup
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## Schedule

**In order to perform the data cleaning of this dataset, we will proceed with the following steps:**

1. Check primary key integrity
2. Check for duplicates
3. Check for missing values
4. Check and handle outliers
5. Check data types and modify accordingly if necessary
6. Save dataset into a new csv file, cleaned

### 0. Loading dataset and fast-reviewing it

In [2]:
customers = pd.read_csv('../data/ml_project1_data.csv')

In [3]:
# Checking dataset shape
print('Num. of customers sampled: ', customers.shape[0], '\nNum. of features observed: ', customers.shape[1])

# Viewing dataset and taking a look at its "face"
customers.head()

Num. of customers sampled:  2240 
Num. of features observed:  29


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0


In [4]:
# Making a (quick) sense of variables distributions
customers.describe(include = 'all')

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2240,2240,2216.000000,2240.000000,2240.000000,2240,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
unique,NaN,NaN,5,8,NaN,NaN,NaN,663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Graduation,Married,NaN,NaN,NaN,2012-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,1127,864,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5592.159821,1968.805804,NaN,NaN,52247.251354,0.444196,0.506250,NaN,49.109375,303.935714,26.302232,166.950000,37.525446,27.062946,44.021875,2.325000,4.084821,2.662054,5.790179,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,3246.662198,11.984069,NaN,NaN,25173.076661,0.538398,0.544538,NaN,28.962453,336.597393,39.773434,225.715373,54.628979,41.280498,52.167439,1.932238,2.778714,2.923101,3.250958,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,0.000000,1893.000000,NaN,NaN,1730.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2828.250000,1959.000000,NaN,NaN,35303.000000,0.000000,0.000000,NaN,24.000000,23.750000,1.000000,16.000000,3.000000,1.000000,9.000000,1.000000,2.000000,0.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5458.500000,1970.000000,NaN,NaN,51381.500000,0.000000,0.000000,NaN,49.000000,173.500000,8.000000,67.000000,12.000000,8.000000,24.000000,2.000000,4.000000,2.000000,5.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8427.750000,1977.000000,NaN,NaN,68522.000000,1.000000,1.000000,NaN,74.000000,504.250000,33.000000,232.000000,50.000000,33.000000,56.000000,3.000000,6.000000,4.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000


## 1. Primary key integrity

In [5]:
# Checking for unique customers' IDs
customers['ID'].duplicated().sum()

0

There are no duplicated primay keys in the dataset. All ID entries are unique, and therefore its integrity holds.

We should move fowards, but we can observe that there is a `Dt_Customer` column and according to the data dictonary provided, it represents the date when the customers joined the database, that is when the customer became a _client_.

A good idea would be also checking the integrity of the `ID` column based on the entry date of the customer in the database. If they both match, the asceding order of `ID` equals the ascending order of `Dt_Customer`.

How can we achieve that?

- Sort the dataset by `ID` and keep a list of the dataset indexes with this sorting;
- Sort the dataset by `Dt_Customer` and keep a list of the dataset indexes when sorted this way;
- Compare if Python's positional indexes of the list are the same for the two Dataset indexes lists gerenated by sorting each of the columns.

In [6]:
# Checking if index sorted by ID is equal to index sorted by 'Dt_Customer'
customers.sort_values(by = 'ID').index.all == \
    customers.sort_values(by = 'Dt_Customer').index.all

False

The orders of the dataset indexes when sorted by the two columns individually are not the same (`FALSE`). Either new customers are being assigned random numbers when subscribed to the database or, worst, new customers are being reasigned old `ID` from customers who might have left.

While the latter is much more problematic than first — we wouldn't be able to perform a _correct_ temporal analysis —, the first is also a fault because it do disregards the oportunity of the `ID` itself being informative (a client numbered 5XXX would be obviously more recent than a client 2XXX).

In this way, we recomend a contacting the Data Engineering department to further investigate.

This configuration of the dataset won't impact this analysis though, and we should carry on checking for duplicates in the whole dataset.

## 2. Checking for duplicates

In [7]:
# Checking duplicated entries for all features
customers.duplicated().sum()

0

There are no full duplicated entries. We can move forward.

## 3. Check for missing values

In [8]:
# Check missing values in each of the features
customers.isna().sum()[customers.isna().sum() > 0]

Income    24
dtype: int64

There are 24 entries with `Income` missing. Let's check how much (in %) this amount represents in the total dataset:

In [9]:
# Missing values as percentage of the whole dataset
print('Ther is aprox.', round((customers.isna().sum()[customers.isna().sum() > 0] / \
                                 customers.shape[0] * 100)['Income'], 2), '% missing Income values in the dataset.')

Ther is aprox. 1.07 % missing Income values in the dataset.


The `Income` variable has 24 missing values, 1,07% of its total.

While this is not relevant, we can still investigate and check if these entries can be recovered.

Let's start by visualizing the entries with missing values in the dataset:

In [10]:
missing_income_df = customers[customers['Income'].isna() == True]
missing_income_df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
10,1994,1983,Graduation,Married,NaN,1,0,2013-11-15,11,5,5,6,0,2,1,1,1,0,2,7,0,0,0,0,0,0,3,11,0
27,5255,1986,Graduation,Single,NaN,1,0,2013-02-20,19,5,1,3,3,263,362,0,27,0,0,1,0,0,0,0,0,0,3,11,0
43,7281,1959,PhD,Single,NaN,0,0,2013-11-05,80,81,11,50,3,2,39,1,1,3,4,2,0,0,0,0,0,0,3,11,0
48,7244,1951,Graduation,Single,NaN,2,1,2014-01-01,96,48,5,48,6,10,7,3,2,1,4,6,0,0,0,0,0,0,3,11,0
58,8557,1982,Graduation,Single,NaN,1,0,2013-06-17,57,11,3,22,2,2,6,2,2,0,3,6,0,0,0,0,0,0,3,11,0


Most common techiniques of imputing values to missing data can be achieved using scikit-learn's `SimpleImputer`. With such imputer, one can choose the `strategy` parameter such as mean, most frequent, etc...

We haven't performed any predictions yet, but we can speculate that `Income` is a strong candidate for a meaninful predictor when analyzing the selling of a new product, such as our case in this analysis. 

Thus, simply imputing the mean, or the most frequent value, could flatten the nuances of such variable and, mainly, could really go wrong in case of outliers. And we do have outliers.

Another option is using `KNNImputer`. We can have a brief explanation of this algorithm from its [documentation](https://scikit-learn.org/stable/modules/impute.html#knnimpute):

> The KNNImputer class provides imputation for filling in missing values using the k-Nearest Neighbors approach. (...) Each missing feature is imputed using values from n_neighbors nearest neighbors that have a value for the feature. The feature of the neighbors are averaged uniformly or weighted by distance to each neighbor.

It sound like a good option for this task! 

But since we will be using it in a dataset with mixed categorical and numerical data, and such numerical data have important differences in orders of magnitude, we need to prepare it before implementing. Specilly for the case of a distance-based algorithm, like KNN. For that, we will be perform the following steps:

- Select variables of interest for the procedure (we will be leaving out `ID`, for its lack of predictive information, and `Dt_Customer`, since it would require another extra steps and we are not sure how much it would represent for the prediction);
- Transform categorical variables into numerical boolean ones, with `pandas` user friendly version of one hot encoding: `get_dummies`;
- Scale the numerical features with `RobustScaler` (Robust Scaler is, well, _robust_ against outliers...);
- Instanciate and apply the `KNNImputer` into scaled features;
- Recover real (monetary units, MU) values from scaled imputed values with `inverse_transform`.

#### i. Selecting variables of interest

In [11]:
# Selecting variables of interest for KNN Imputer
var_selection_imputing = [
#     'ID', 
    'Year_Birth', 
    'Education', 
    'Marital_Status', 
    'Income', 
    'Kidhome',
    'Teenhome', 
#     'Dt_Customer', 
    'Recency', 
    'MntWines', 
    'MntFruits',
    'MntMeatProducts', 
    'MntFishProducts', 
    'MntSweetProducts',
    'MntGoldProds', 
    'NumDealsPurchases', 
    'NumWebPurchases',
    'NumCatalogPurchases', 
    'NumStorePurchases', 
    'NumWebVisitsMonth',
    'AcceptedCmp3', 
    'AcceptedCmp4', 
    'AcceptedCmp5', 
    'AcceptedCmp1',
    'AcceptedCmp2', 
    'Complain', 
    'Z_CostContact', 
    'Z_Revenue', 
    'Response'
]

#### ii. One Hote Enconding (with `get_dummies`)

In [12]:
# Transforming categorical variables to numerical boolean ones
customers_dummies = pd.get_dummies(customers[var_selection_imputing])
customers_dummies.head()

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Education_2n Cycle,Education_Basic,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,1957,58138.0,0,0,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1,0,0,1,0,0,0,0,0,0,1,0,0,0
1,1954,46344.0,1,1,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,1965,71613.0,0,0,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0,0,0,1,0,0,0,0,0,0,0,1,0,0
3,1984,26646.0,1,0,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,1981,58293.0,1,0,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0,0,0,0,0,1,0,0,0,1,0,0,0,0


#### iii. Scaling variables to soften magnitude differences

In [13]:
robust_scaler = RobustScaler()

robust_scaler.fit(customers_dummies)

customers_dummies_scaled = pd.DataFrame(robust_scaler.transform(customers_dummies), 
                                        columns = customers_dummies.columns)

customers_dummies_scaled.head()

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Education_2n Cycle,Education_Basic,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,-0.722222,0.203393,0.0,0.0,0.18,0.960458,2.50000,2.217593,3.404255,2.50000,1.361702,0.5,1.00,2.00,-0.2,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.888889,-0.151645,1.0,1.0,-0.22,-0.338189,-0.21875,-0.282407,-0.212766,-0.21875,-0.382979,0.0,-0.75,-0.25,-0.6,-0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.277778,0.609034,0.0,0.0,-0.46,0.525494,1.28125,0.277778,2.106383,0.40625,0.382979,-0.5,1.00,0.00,1.0,-0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.777778,-0.744619,1.0,0.0,-0.46,-0.338189,-0.12500,-0.217593,-0.042553,-0.15625,-0.404255,0.0,-0.50,-0.50,-0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.611111,0.208059,1.0,0.0,0.90,-0.001041,1.09375,0.236111,0.723404,0.59375,-0.191489,1.5,0.25,0.25,0.2,-0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### iv. KNNImputer for missing values of `Income` 

In [89]:
# Instanciating the KNN Imuputer
knn_imputer = KNNImputer(n_neighbors = 5, weights = 'distance')
customer_dummies_scaled_imputed = pd.DataFrame(knn_imputer.fit_transform(customers_dummies_scaled),
                                               columns = customers_dummies_scaled.columns)

customer_dummies_scaled_imputed.head()

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Education_2n Cycle,Education_Basic,Education_Graduation,Education_Master,Education_PhD,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO
0,-0.722222,0.203393,0.0,0.0,0.18,0.960458,2.50000,2.217593,3.404255,2.50000,1.361702,0.5,1.00,2.00,-0.2,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.888889,-0.151645,1.0,1.0,-0.22,-0.338189,-0.21875,-0.282407,-0.212766,-0.21875,-0.382979,0.0,-0.75,-0.25,-0.6,-0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.277778,0.609034,0.0,0.0,-0.46,0.525494,1.28125,0.277778,2.106383,0.40625,0.382979,-0.5,1.00,0.00,1.0,-0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.777778,-0.744619,1.0,0.0,-0.46,-0.338189,-0.12500,-0.217593,-0.042553,-0.15625,-0.404255,0.0,-0.50,-0.50,-0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.611111,0.208059,1.0,0.0,0.90,-0.001041,1.09375,0.236111,0.723404,0.59375,-0.191489,1.5,0.25,0.25,0.2,-0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### v. Getting (MU) values from scaled imputed values

In [99]:
# Getting income restore to MU value
income_restored = pd.Series(robust_scaler.inverse_transform(customer_dummies_scaled_imputed)[:, 1])
income_restored

0       58138.0
1       46344.0
2       71613.0
3       26646.0
4       58293.0
         ...   
2235    61223.0
2236    64014.0
2237    56981.0
2238    69245.0
2239    52869.0
Length: 2240, dtype: float64

In [100]:
# Replacing Income missing data with new KNNImputer Income Data
customers.loc[missing_income_df.index, 'Income'] = round(income_restored, 0)
customers.loc[missing_income_df.index, :].head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
10,1994,1983,Graduation,Married,27686.0,1,0,2013-11-15,11,5,5,6,0,2,1,1,1,0,2,7,0,0,0,0,0,0,3,11,0
27,5255,1986,Graduation,Single,58899.0,1,0,2013-02-20,19,5,1,3,3,263,362,0,27,0,0,1,0,0,0,0,0,0,3,11,0
43,7281,1959,PhD,Single,43196.0,0,0,2013-11-05,80,81,11,50,3,2,39,1,1,3,4,2,0,0,0,0,0,0,3,11,0
48,7244,1951,Graduation,Single,33421.0,2,1,2014-01-01,96,48,5,48,6,10,7,3,2,1,4,6,0,0,0,0,0,0,3,11,0
58,8557,1982,Graduation,Single,28567.0,1,0,2013-06-17,57,11,3,22,2,2,6,2,2,0,3,6,0,0,0,0,0,0,3,11,0


In [105]:
# Checking or missing values after imputation
customers.isna().sum()[customers.isna().sum() > 0].any()

False

No more values missing in the dataset. We can move forward to the next step.

## 5. Handling outliers

In [ ]:

plt.figure(figsize = (16, 10))
for i, feature in enumerate(features_numerical):
    plt.subplot(3, 3, i + 1)
    data_cleaned[feature].plot(kind = 'box')
    plt.title(feature)

In [ ]:
customers.head()

In [ ]:
customers.describe()

In [ ]:
sns.distplot(customers['Income']);

In [ ]:
customers[customers['Income'] > 300000]

In [ ]:
customers['Dt_Customer'].min(), customers['Dt_Customer'].max()